In [1]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from FAB3 import *

learning_rate = 0.001  
gamma = 1
buffer_limit = 10000
batch_size = 16

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(64,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 9)
        else:
            return out.argmax().item()
        
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        return out.argmax().item(),out
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 10
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        #여기는 sample_action 구간
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
            
        #학습구간    
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)
        
        #결과 및 파라미터 저장    
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            params = q.state_dict()
            param_name = str(n_epi)+"q2.pt"
            #print(param_name)
            torch.save(params, param_name)
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        
        #여기는 select_action 구간
        s = env.reset()
        done = False
        score = 0.0
        params = q.state_dict()
        torch.save(params, "q2.pt" )
        while not done:
            a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
            #print(a_list)
            #print(a)
            s_prime, r, done = env.step(a)
            #print(r)
            s = s_prime
            score += r
            if done:
                break
        Flow_time, machine_utill ,util, makespan = env.performance_measure()
        print("--------------------------------------------------")
        print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
        print("n_epi: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),0))
        
        
        
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
    
    
    s = env.reset()
    done = False
    score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        #print(a_list)
        #print(a)
        s_prime, r, done = env.step(a)
        #print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    #env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
for i in range(1):
    Flow_time, machine_util, util, makespan, score =main()
    print("FlowTime:" , Flow_time)
    print("machine_util:" , machine_util)
    print("util:" , util)
    print("makespan:" , makespan)
    print("Score" , score)
    
    
params = torch.load("8q2.pt")
q = Qnet()
q.load_state_dict(params)
q.eval()
env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM8_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1) 
s = env.reset()
done = False
score = 0.0
epsilon = max(0.01 , 0.08 - 0.02*(20/200))
while not done:
    a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
    #print(a_list)
    #print(a)
    s_prime, r, done = env.step(a)
    #print(r)
    s = s_prime
    score += r
    if done:
        break
Flow_time, machine_util, util, makespan = env.performance_measure()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)    

--------------------------------------------------
flow time: 26423, util : 0.956, makespan : 875
n_epi: 0, score : -10904.0, n_buffer : 340, eps : 0.0%
--------------------------------------------------
flow time: 26414, util : 0.944, makespan : 837
n_episode: 1, score : -10073.0, n_buffer : 680, eps : 8.0%
--------------------------------------------------
flow time: 26423, util : 0.956, makespan : 875
n_epi: 1, score : -10904.0, n_buffer : 680, eps : 0.0%
--------------------------------------------------
flow time: 27227, util : 0.945, makespan : 866
n_episode: 2, score : -10516.0, n_buffer : 1020, eps : 8.0%
--------------------------------------------------
flow time: 26423, util : 0.956, makespan : 875
n_epi: 2, score : -10904.0, n_buffer : 1020, eps : 0.0%
--------------------------------------------------
flow time: 26464, util : 0.953, makespan : 866
n_episode: 3, score : -10219.0, n_buffer : 1360, eps : 8.0%
--------------------------------------------------
flow time: 26423

<ipython-input-1-b0d05595fc02>:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)


--------------------------------------------------
flow time: 27462, util : 0.944, makespan : 852
n_episode: 5, score : -10599.0, n_buffer : 2040, eps : 8.0%
--------------------------------------------------
flow time: 29751, util : 0.959, makespan : 842
n_epi: 5, score : -10191.0, n_buffer : 2040, eps : 0.0%
--------------------------------------------------
flow time: 29322, util : 0.941, makespan : 873
n_episode: 6, score : -10579.0, n_buffer : 2380, eps : 7.9%
--------------------------------------------------
flow time: 32858, util : 0.932, makespan : 858
n_epi: 6, score : -10344.0, n_buffer : 2380, eps : 0.0%
--------------------------------------------------
flow time: 34843, util : 0.918, makespan : 885
n_episode: 7, score : -10586.0, n_buffer : 2720, eps : 7.9%
--------------------------------------------------
flow time: 57875, util : 0.844, makespan : 954
n_epi: 7, score : -11428.0, n_buffer : 2720, eps : 0.0%
--------------------------------------------------
flow time: 53

--------------------------------------------------
flow time: 24710, util : 0.905, makespan : 738
n_epi: 31, score : -8574.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 25834, util : 0.893, makespan : 770
n_episode: 32, score : -8853.0, n_buffer : 10000, eps : 7.7%
--------------------------------------------------
flow time: 25079, util : 0.960, makespan : 848
n_epi: 32, score : -10576.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 26329, util : 0.952, makespan : 866
n_episode: 33, score : -10831.0, n_buffer : 10000, eps : 7.7%
--------------------------------------------------
flow time: 25079, util : 0.960, makespan : 848
n_epi: 33, score : -10576.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 27120, util : 0.943, makespan : 833
n_episode: 34, score : -10255.0, n_buffer : 10000, eps : 7.7%
--------------------------------------------------
flo

--------------------------------------------------
flow time: 29079, util : 0.914, makespan : 727
n_episode: 58, score : -8891.0, n_buffer : 10000, eps : 7.4%
--------------------------------------------------
flow time: 32078, util : 0.891, makespan : 667
n_epi: 58, score : -7918.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 33899, util : 0.886, makespan : 673
n_episode: 59, score : -8277.0, n_buffer : 10000, eps : 7.4%
--------------------------------------------------
flow time: 26171, util : 0.921, makespan : 664
n_epi: 59, score : -7855.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 27601, util : 0.905, makespan : 685
n_episode: 60, score : -8402.0, n_buffer : 10000, eps : 7.4%
--------------------------------------------------
flow time: 26557, util : 0.918, makespan : 661
n_epi: 60, score : -7821.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow ti

...

--------------------------------------------------
flow time: 24405, util : 0.938, makespan : 783
n_epi: 370, score : -9975.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 25174, util : 0.930, makespan : 775
n_episode: 371, score : -9382.0, n_buffer : 10000, eps : 4.3%
--------------------------------------------------
flow time: 49980, util : 0.855, makespan : 885
n_epi: 371, score : -10541.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 49091, util : 0.858, makespan : 884
n_episode: 372, score : -10634.0, n_buffer : 10000, eps : 4.3%
--------------------------------------------------
flow time: 27058, util : 0.918, makespan : 795
n_epi: 372, score : -9146.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 28907, util : 0.916, makespan : 764
n_episode: 373, score : -9058.0, n_buffer : 10000, eps : 4.3%
--------------------------------------------------

--------------------------------------------------
flow time: 23894, util : 0.928, makespan : 738
n_epi: 396, score : -9098.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 24651, util : 0.924, makespan : 730
n_episode: 397, score : -8793.0, n_buffer : 10000, eps : 4.0%
--------------------------------------------------
flow time: 50593, util : 0.858, makespan : 892
n_epi: 397, score : -10544.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 48722, util : 0.857, makespan : 889
n_episode: 398, score : -10575.0, n_buffer : 10000, eps : 4.0%
--------------------------------------------------
flow time: 24802, util : 0.949, makespan : 731
n_epi: 398, score : -8736.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 26090, util : 0.935, makespan : 777
n_episode: 399, score : -9055.0, n_buffer : 10000, eps : 4.0%
--------------------------------------------------

--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_epi: 422, score : -8440.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 24621, util : 0.938, makespan : 727
n_episode: 423, score : -8920.0, n_buffer : 10000, eps : 3.8%
--------------------------------------------------
flow time: 24736, util : 0.907, makespan : 721
n_epi: 423, score : -8453.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 24885, util : 0.903, makespan : 728
n_episode: 424, score : -8445.0, n_buffer : 10000, eps : 3.8%
--------------------------------------------------
flow time: 26102, util : 0.916, makespan : 721
n_epi: 424, score : -8435.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 27023, util : 0.915, makespan : 737
n_episode: 425, score : -8870.0, n_buffer : 10000, eps : 3.8%
--------------------------------------------------
f

--------------------------------------------------
flow time: 28725, util : 0.890, makespan : 686
n_epi: 448, score : -8261.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 29678, util : 0.884, makespan : 689
n_episode: 449, score : -8350.0, n_buffer : 10000, eps : 3.5%
--------------------------------------------------
flow time: 26011, util : 0.928, makespan : 795
n_epi: 449, score : -9558.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 27390, util : 0.922, makespan : 804
n_episode: 450, score : -9776.0, n_buffer : 10000, eps : 3.5%
--------------------------------------------------
flow time: 27116, util : 0.889, makespan : 729
n_epi: 450, score : -8649.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 26929, util : 0.884, makespan : 724
n_episode: 451, score : -8576.0, n_buffer : 10000, eps : 3.5%
--------------------------------------------------
f

KeyboardInterrupt: 

In [ ]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from FAB3 import *

learning_rate = 0.0001  
gamma = 0.98
buffer_limit = 100000
batch_size = 64

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(64,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 9)
        else:
            return out.argmax().item()
        
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        return out.argmax().item(),out
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        #여기는 sample_action 구간
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
            
        #학습구간    
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)
        
        #결과 및 파라미터 저장    
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            params = q.state_dict()
            param_name = str(n_epi)+"q2.pt"
            #print(param_name)
            torch.save(params, param_name)
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        
        #여기는 select_action 구간
        s = env.reset()
        done = False
        score = 0.0
        params = q.state_dict()
        torch.save(params, "q2.pt" )
        while not done:
            a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
            #print(a_list)
            #print(a)
            s_prime, r, done = env.step(a)
            #print(r)
            s = s_prime
            score += r
            if done:
                break
        Flow_time, machine_utill ,util, makespan = env.performance_measure()
        print("--------------------------------------------------")
        print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
        print("n_epi: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),0))
        
        
        
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
    
    
    s = env.reset()
    done = False
    score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        #print(a_list)
        #print(a)
        s_prime, r, done = env.step(a)
        #print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    #env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
for i in range(1):
    Flow_time, machine_util, util, makespan, score =main()
    print("FlowTime:" , Flow_time)
    print("machine_util:" , machine_util)
    print("util:" , util)
    print("makespan:" , makespan)
    print("Score" , score)
    
    
params = torch.load("8q2.pt")
q = Qnet()
q.load_state_dict(params)
q.eval()
env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM8_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1) 
s = env.reset()
done = False
score = 0.0
epsilon = max(0.01 , 0.08 - 0.02*(20/200))
while not done:
    a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
    #print(a_list)
    #print(a)
    s_prime, r, done = env.step(a)
    #print(r)
    s = s_prime
    score += r
    if done:
        break
Flow_time, machine_util, util, makespan = env.performance_measure()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)    

--------------------------------------------------
flow time: 48106, util : 0.940, makespan : 1013
n_epi: 0, score : -11642.0, n_buffer : 340, eps : 0.0%
--------------------------------------------------
flow time: 48314, util : 0.931, makespan : 1016
n_episode: 1, score : -11699.0, n_buffer : 680, eps : 8.0%
--------------------------------------------------
flow time: 48106, util : 0.940, makespan : 1013
n_epi: 1, score : -11642.0, n_buffer : 680, eps : 0.0%
--------------------------------------------------
flow time: 49063, util : 0.924, makespan : 1015
n_episode: 2, score : -11777.0, n_buffer : 1020, eps : 8.0%
--------------------------------------------------
flow time: 48106, util : 0.940, makespan : 1013
n_epi: 2, score : -11642.0, n_buffer : 1020, eps : 0.0%
--------------------------------------------------
flow time: 47947, util : 0.924, makespan : 1009
n_episode: 3, score : -11628.0, n_buffer : 1360, eps : 8.0%
--------------------------------------------------
flow time:

--------------------------------------------------
flow time: 30506, util : 0.943, makespan : 837
n_episode: 27, score : -10280.0, n_buffer : 9520, eps : 7.7%
--------------------------------------------------
flow time: 30362, util : 0.954, makespan : 817
n_epi: 27, score : -9988.0, n_buffer : 9520, eps : 0.0%
--------------------------------------------------
flow time: 32072, util : 0.936, makespan : 834
n_episode: 28, score : -10124.0, n_buffer : 9860, eps : 7.7%
--------------------------------------------------
flow time: 32003, util : 0.950, makespan : 848
n_epi: 28, score : -10010.0, n_buffer : 9860, eps : 0.0%
--------------------------------------------------
flow time: 33521, util : 0.928, makespan : 874
n_episode: 29, score : -10693.0, n_buffer : 10200, eps : 7.7%
--------------------------------------------------
flow time: 34001, util : 0.940, makespan : 841
n_epi: 29, score : -10125.0, n_buffer : 10200, eps : 0.0%
--------------------------------------------------
flow t

--------------------------------------------------
flow time: 37029, util : 0.903, makespan : 906
n_episode: 53, score : -10738.0, n_buffer : 18360, eps : 7.5%
--------------------------------------------------
flow time: 36093, util : 0.920, makespan : 830
n_epi: 53, score : -9741.0, n_buffer : 18360, eps : 0.0%
--------------------------------------------------
flow time: 35956, util : 0.917, makespan : 853
n_episode: 54, score : -10242.0, n_buffer : 18700, eps : 7.5%
--------------------------------------------------
flow time: 35985, util : 0.921, makespan : 853
n_epi: 54, score : -9996.0, n_buffer : 18700, eps : 0.0%
--------------------------------------------------
flow time: 38058, util : 0.911, makespan : 873
n_episode: 55, score : -10225.0, n_buffer : 19040, eps : 7.4%
--------------------------------------------------
flow time: 35911, util : 0.899, makespan : 834
n_epi: 55, score : -9866.0, n_buffer : 19040, eps : 0.0%
--------------------------------------------------
flow

--------------------------------------------------
flow time: 27538, util : 0.899, makespan : 753
n_epi: 79, score : -8792.0, n_buffer : 27200, eps : 0.0%
--------------------------------------------------
flow time: 31339, util : 0.881, makespan : 813
n_episode: 80, score : -9782.0, n_buffer : 27540, eps : 7.2%
--------------------------------------------------
flow time: 25768, util : 0.918, makespan : 707
n_epi: 80, score : -8303.0, n_buffer : 27540, eps : 0.0%
--------------------------------------------------
flow time: 28473, util : 0.897, makespan : 720
n_episode: 81, score : -8926.0, n_buffer : 27880, eps : 7.2%
--------------------------------------------------
flow time: 23970, util : 0.938, makespan : 721
n_epi: 81, score : -8377.0, n_buffer : 27880, eps : 0.0%
--------------------------------------------------
flow time: 24395, util : 0.925, makespan : 738
n_episode: 82, score : -8735.0, n_buffer : 28220, eps : 7.2%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 27647, util : 0.899, makespan : 816
n_episode: 106, score : -9965.0, n_buffer : 36380, eps : 6.9%
--------------------------------------------------
flow time: 27180, util : 0.875, makespan : 728
n_epi: 106, score : -8667.0, n_buffer : 36380, eps : 0.0%
--------------------------------------------------
flow time: 28063, util : 0.886, makespan : 749
n_episode: 107, score : -9093.0, n_buffer : 36720, eps : 6.9%
--------------------------------------------------
flow time: 36324, util : 0.886, makespan : 662
n_epi: 107, score : -7886.0, n_buffer : 36720, eps : 0.0%
--------------------------------------------------
flow time: 38420, util : 0.873, makespan : 676
n_episode: 108, score : -8319.0, n_buffer : 37060, eps : 6.9%
--------------------------------------------------
flow time: 37579, util : 0.891, makespan : 659
n_epi: 108, score : -7920.0, n_buffer : 37060, eps : 0.0%
--------------------------------------------------
f

--------------------------------------------------
flow time: 32169, util : 0.865, makespan : 705
n_episode: 132, score : -8674.0, n_buffer : 45220, eps : 6.7%
--------------------------------------------------
flow time: 31467, util : 0.875, makespan : 686
n_epi: 132, score : -8131.0, n_buffer : 45220, eps : 0.0%
--------------------------------------------------
flow time: 32200, util : 0.864, makespan : 704
n_episode: 133, score : -8663.0, n_buffer : 45560, eps : 6.7%
--------------------------------------------------
flow time: 32306, util : 0.874, makespan : 684
n_epi: 133, score : -8148.0, n_buffer : 45560, eps : 0.0%
--------------------------------------------------
flow time: 33129, util : 0.873, makespan : 689
n_episode: 134, score : -8332.0, n_buffer : 45900, eps : 6.7%
--------------------------------------------------
flow time: 24711, util : 0.906, makespan : 738
n_epi: 134, score : -8575.0, n_buffer : 45900, eps : 0.0%
--------------------------------------------------
f

In [1]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from FAB3 import *

learning_rate = 0.0001  
gamma = 0.98
buffer_limit = 100000
batch_size = 64

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(64,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 9)
        else:
            return out.argmax().item()
        
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        return out.argmax().item(),out
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1)
    q = Qnet()
    params = torch.load("154q2.pt")
    q.load_state_dict(params)
    q.eval()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(154, 1000):
        #여기는 sample_action 구간
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
            
        #학습구간    
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)
        
        #결과 및 파라미터 저장    
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            params = q.state_dict()
            param_name = str(n_epi)+"q2.pt"
            #print(param_name)
            torch.save(params, param_name)
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        
        #여기는 select_action 구간
        s = env.reset()
        done = False
        score = 0.0
        params = q.state_dict()
        torch.save(params, "q2.pt" )
        while not done:
            a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
            #print(a_list)
            #print(a)
            s_prime, r, done = env.step(a)
            #print(r)
            s = s_prime
            score += r
            if done:
                break
        Flow_time, machine_utill ,util, makespan = env.performance_measure()
        print("--------------------------------------------------")
        print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
        print("n_epi: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),0))
        
        
        
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
    
    
    s = env.reset()
    done = False
    score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        #print(a_list)
        #print(a)
        s_prime, r, done = env.step(a)
        #print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    #env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
for i in range(1):
    Flow_time, machine_util, util, makespan, score =main()
    print("FlowTime:" , Flow_time)
    print("machine_util:" , machine_util)
    print("util:" , util)
    print("makespan:" , makespan)
    print("Score" , score)
    
    
params = torch.load("8q2.pt")
q = Qnet()
q.load_state_dict(params)
q.eval()
env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM8_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1) 
s = env.reset()
done = False
score = 0.0
epsilon = max(0.01 , 0.08 - 0.02*(20/200))
while not done:
    a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
    #print(a_list)
    #print(a)
    s_prime, r, done = env.step(a)
    #print(r)
    s = s_prime
    score += r
    if done:
        break
Flow_time, machine_util, util, makespan = env.performance_measure()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)    

--------------------------------------------------
flow time: 27181, util : 0.888, makespan : 702
n_episode: 154, score : -8610.0, n_buffer : 340, eps : 6.5%
--------------------------------------------------
flow time: 24748, util : 0.907, makespan : 701
n_epi: 154, score : -8198.0, n_buffer : 340, eps : 0.0%
--------------------------------------------------
flow time: 25461, util : 0.896, makespan : 719
n_episode: 155, score : -8522.0, n_buffer : 680, eps : 6.5%
--------------------------------------------------
flow time: 24748, util : 0.907, makespan : 701
n_epi: 155, score : -8198.0, n_buffer : 680, eps : 0.0%
--------------------------------------------------
flow time: 25531, util : 0.902, makespan : 726
n_episode: 156, score : -8697.0, n_buffer : 1020, eps : 6.4%
--------------------------------------------------
flow time: 24748, util : 0.907, makespan : 701
n_epi: 156, score : -8198.0, n_buffer : 1020, eps : 0.0%
--------------------------------------------------
flow time: 

<ipython-input-1-a9d4af9cae48>:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)


--------------------------------------------------
flow time: 25603, util : 0.906, makespan : 741
n_episode: 159, score : -8712.0, n_buffer : 2040, eps : 6.4%
--------------------------------------------------
flow time: 29251, util : 0.910, makespan : 674
n_epi: 159, score : -7939.0, n_buffer : 2040, eps : 0.0%
--------------------------------------------------
flow time: 29247, util : 0.905, makespan : 699
n_episode: 160, score : -8686.0, n_buffer : 2380, eps : 6.4%
--------------------------------------------------
flow time: 31966, util : 0.876, makespan : 768
n_epi: 160, score : -9144.0, n_buffer : 2380, eps : 0.0%
--------------------------------------------------
flow time: 35916, util : 0.884, makespan : 761
n_episode: 161, score : -9316.0, n_buffer : 2720, eps : 6.4%
--------------------------------------------------
flow time: 31976, util : 0.893, makespan : 828
n_epi: 161, score : -9903.0, n_buffer : 2720, eps : 0.0%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 24765, util : 0.912, makespan : 733
n_epi: 185, score : -8726.0, n_buffer : 10880, eps : 0.0%
--------------------------------------------------
flow time: 25552, util : 0.902, makespan : 756
n_episode: 186, score : -8991.0, n_buffer : 11220, eps : 6.1%
--------------------------------------------------
flow time: 24427, util : 0.930, makespan : 723
n_epi: 186, score : -8492.0, n_buffer : 11220, eps : 0.0%
--------------------------------------------------
flow time: 25627, util : 0.920, makespan : 755
n_episode: 187, score : -9040.0, n_buffer : 11560, eps : 6.1%
--------------------------------------------------
flow time: 24442, util : 0.911, makespan : 731
n_epi: 187, score : -8295.0, n_buffer : 11560, eps : 0.0%
--------------------------------------------------
flow time: 25868, util : 0.902, makespan : 704
n_episode: 188, score : -8483.0, n_buffer : 11900, eps : 6.1%
--------------------------------------------------
f

--------------------------------------------------
flow time: 23473, util : 0.946, makespan : 754
n_epi: 211, score : -8209.0, n_buffer : 19720, eps : 0.0%
--------------------------------------------------
flow time: 24706, util : 0.933, makespan : 723
n_episode: 212, score : -8606.0, n_buffer : 20060, eps : 5.9%
--------------------------------------------------
flow time: 23473, util : 0.946, makespan : 754
n_epi: 212, score : -8209.0, n_buffer : 20060, eps : 0.0%
--------------------------------------------------
flow time: 24384, util : 0.930, makespan : 731
n_episode: 213, score : -8477.0, n_buffer : 20400, eps : 5.9%
--------------------------------------------------
flow time: 23811, util : 0.936, makespan : 733
n_epi: 213, score : -8493.0, n_buffer : 20400, eps : 0.0%
--------------------------------------------------
flow time: 25638, util : 0.917, makespan : 707
n_episode: 214, score : -8260.0, n_buffer : 20740, eps : 5.9%
--------------------------------------------------
f

...

--------------------------------------------------
flow time: 23960, util : 0.909, makespan : 717
n_epi: 887, score : -8503.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23946, util : 0.912, makespan : 733
n_episode: 888, score : -8566.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24005, util : 0.916, makespan : 716
n_epi: 888, score : -8284.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24060, util : 0.913, makespan : 713
n_episode: 889, score : -8204.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23830, util : 0.936, makespan : 733
n_epi: 889, score : -8418.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24196, util : 0.935, makespan : 744
n_episode: 890, score : -8193.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 26878, util : 0.906, makespan : 731
n_epi: 913, score : -8523.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 25944, util : 0.910, makespan : 724
n_episode: 914, score : -8224.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24037, util : 0.925, makespan : 687
n_epi: 914, score : -8186.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24515, util : 0.920, makespan : 755
n_episode: 915, score : -8433.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23946, util : 0.939, makespan : 705
n_epi: 915, score : -8335.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23182, util : 0.940, makespan : 713
n_episode: 916, score : -8252.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 24246, util : 0.909, makespan : 748
n_epi: 939, score : -8349.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24864, util : 0.910, makespan : 738
n_episode: 940, score : -8409.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_epi: 940, score : -8440.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23530, util : 0.941, makespan : 719
n_episode: 941, score : -8569.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24356, util : 0.912, makespan : 712
n_epi: 941, score : -8243.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24684, util : 0.919, makespan : 744
n_episode: 942, score : -8349.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 23885, util : 0.918, makespan : 727
n_epi: 965, score : -8330.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23869, util : 0.918, makespan : 720
n_episode: 966, score : -8352.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23946, util : 0.939, makespan : 705
n_epi: 966, score : -8335.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24085, util : 0.930, makespan : 733
n_episode: 967, score : -8226.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24246, util : 0.909, makespan : 748
n_epi: 967, score : -8349.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 25054, util : 0.918, makespan : 756
n_episode: 968, score : -8513.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 26437, util : 0.896, makespan : 740
n_epi: 991, score : -8586.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 26434, util : 0.903, makespan : 737
n_episode: 992, score : -8428.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24315, util : 0.933, makespan : 707
n_epi: 992, score : -8249.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24422, util : 0.931, makespan : 714
n_episode: 993, score : -8099.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23995, util : 0.925, makespan : 759
n_epi: 993, score : -8237.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24607, util : 0.911, makespan : 746
n_episode: 994, score : -8580.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------